# 🔍 Exploración y Entrenamiento Inicial del Modelo ESI
Este notebook carga el dataset sintético de triage ESI, lo explora, y entrena un modelo Random Forest para predecir el nivel de triage.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

# Cargar el dataset
df = pd.read_csv('../data/raw/dataset_triage_ESI_500.csv')

# Ver estructura general
print(df.info())
df.head()

In [ ]:
# Distribución de clases
sns.countplot(data=df, x='ESI')
plt.title('Distribución de clases ESI')
plt.show()

In [ ]:
# Codificación de variables categóricas
df['Riesgo Vital'] = df['Riesgo Vital'].map({'Sí': 1, 'No': 0})
df['Intervención Urgente'] = df['Intervención Urgente'].map({'Sí': 1, 'No': 0})
le = LabelEncoder()
df['Motivo'] = le.fit_transform(df['Motivo'])

In [ ]:
# División de datos
X = df.drop(columns='ESI')
y = df['ESI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Guardar modelo entrenado
joblib.dump(model, '../models/modelo_triage.pkl')